# TensorFlow BYOM: Train locally and deploy on SageMaker.


1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
    1. [Permissions and environment variables](#Permissions-and-environment-variables)
    2. [Model definitions](#Model-definitions)
    3. [Data Setup](#Data-setup)
3. [Training the network locally](#Training)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
    1. [Export from TensorFlow](#Export-the-model-from-tensorflow)
    2. [Import model into SageMaker](#Import-model-into-SageMaker)
    3. [Create endpoint](#Create-endpoint) 
5. [Validate the endpoint for use](#Validate-the-endpoint-for-use)

__Note__: Compare this with the [tensorflow bring your own model example](../tensorflow_iris_byom/tensorflow_BYOM_iris.ipynb)

## Introduction 

This notebook can be compared to [Iris classification example notebook](../tensorflow_iris_dnn_classifier_using_estimators/tensorflow_iris_dnn_classifier_using_estimators.ipynb) in terms of its functionality. We will do the same classification task, but we will train the same network locally in the box from where this notebook is being run. We then setup a real-time hosted endpoint in SageMaker.

Consider the following model definition for IRIS classification. This mode uses the ``tensorflow.estimator.DNNClassifier`` which is a pre-defined estimator module for its model definition. The model definition is the same as the one used in the [Iris classification example notebook](../tensorflow_iris_dnn_classifier_using_estimators/tensorflow_iris_dnn_classifier_using_estimators.ipynb)

## Prequisites and Preprocessing
### Permissions and environment variables

Here we set up the linkage and authentication to AWS services. In this notebook we only need the roles used to give learning and hosting access to your data. The Sagemaker SDK will use S3 defualt buckets when needed. If the ``get_execution_role``  does not return a role with the appropriate permissions, you'll need to specify an IAM role arn that does.

In [1]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

### Model Definitions

We use the [``tensorflow.estimator.DNNClassifier``](https://www.tensorflow.org/api_docs/python/tf/estimator/DNNClassifier) estimator to set up our network. We also need to write some methods for serving inputs during hosting and training. These methods are all found below.

In [2]:
!cat serving_entrypoint.py

import numpy as np
import tensorflow as tf


INPUT_TENSOR_NAME = 'image_tensor:0'


def serving_input_fn():
    inputs = {INPUT_TENSOR_NAME: tf.placeholder(tf.uint8, [None, None, None, 3])}
    return tf.estimator.export.ServingInputReceiver(inputs, inputs)


Create an estimator object with this model definition.

In [ ]:
from iris_dnn_classifier import estimator_fn
classifier = estimator_fn(run_config = None, params = None)

### Data setup

Next, we need to pull the data from tensorflow repository and make them ready for training. The following will code block should do that.

In [ ]:
import os 
from six.moves.urllib.request import urlopen

# Data sets
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

if not os.path.exists(IRIS_TRAINING):
    raw = urlopen(IRIS_TRAINING_URL).read()
    with open(IRIS_TRAINING, "wb") as f:
      f.write(raw)

if not os.path.exists(IRIS_TEST):
    raw = urlopen(IRIS_TEST_URL).read()
    with open(IRIS_TEST, "wb") as f:
      f.write(raw)

Create the data input streamer object.

In [ ]:
from iris_dnn_classifier import train_input_fn
train_func = train_input_fn('.', params = None)

### Training

It is time to train the network. Since we are training the network locally, we can make use of TensorFlow's ``tensorflow.Estimator.train`` method. The model is trained locally in the box.

In [ ]:
classifier.train(input_fn = train_func, steps = 1000)

## Set up hosting for the model

### Export the model from tensorflow

In order to set up hosting, we have to import the model from training to hosting. We will begin by exporting the model from TensorFlow and saving it down. Analogous to the [MXNet example](../mxnet_mnist_byom/mxnet_mnist.ipynb), some structure needs to be followed. The exported model has to be converted into a form that is readable by ``sagemaker.mxnet.model.MXNetModel``. The following code describes exporting the model in a form that does the same:

There is a small difference between a SageMaker model and a TensorFlow model. The conversion is easy and fairly trivial. Simply move the tensorflow exported model into a directory ``export\Servo\`` and tar the entire directory. SageMaker will recognize this as a loadable TensorFlow model.

In [3]:
%%sh
mkdir -p export/Servo && cp -r frozen/saved_model export/Servo/1

In [4]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

### Import model into SageMaker

Open a new sagemaker session and upload the model on to the default S3 bucket. We can use the ``sagemaker.Session.upload_data`` method to do this. We need the location of where we exported the model from MXNet and where in our default bucket we want to store the model(``/model``). The default S3 bucket can be found using the ``sagemaker.Session.default_bucket`` method.

In [5]:
import sagemaker

sagemaker_session = sagemaker.Session()
# inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='manabiya')

Use the ``sagemaker.mxnet.model.TensorFlowModel`` to import the model into SageMaker that can be deployed. We need the location of the S3 bucket where we have the model, the role for authentication and the entry_point where the model defintion is stored (``iris_dnn_classifier.py``). The import call is the following:

In [6]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/manabiya/model.tar.gz',
                                  role = role,
                                  entry_point = 'serving_entrypoint.py')

### Create endpoint

Now the model is ready to be deployed at a SageMaker endpoint. We can use the ``sagemaker.mxnet.model.TensorFlowModel.deploy`` method to do this. Unless you have created or prefer other instances, we recommend using 1 ``'ml.m4.xlarge'`` instance for this example. These are supplied as arguments. 

In [7]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                          instance_type='ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-tensorflow-py2-cpu-2018-03-15-02-28-39-225
INFO:sagemaker:Creating endpoint with name sagemaker-tensorflow-py2-cpu-2018-03-15-02-28-39-225


--------------------------------------------------------------------------------------------------------------!CPU times: user 500 ms, sys: 48 ms, total: 548 ms
Wall time: 9min 35s


### Validate the endpoint for use

We can now use this endpoint to classify. Run an example prediction on a sample to ensure that it works.

In [8]:
from PIL import Image

In [15]:
import numpy  as np

In [9]:
def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

In [12]:
import os, glob

In [13]:
PATH_TO_TEST_IMAGES_DIR = './data-manabiya/width1000/image'
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, '*.png'))

# Size, in inches, of the output images.
IMAGE_SIZE = (12*2, 8*2)

In [16]:
image1 = Image.open(TEST_IMAGE_PATHS[0])
image_np = load_image_into_numpy_array(image1)
image_np_expanded = np.expand_dims(image_np, axis=0)
output_dic = predictor.predict(image_np_expanded)

ClientError: An error occurred (413) when calling the InvokeEndpoint operation: HTTP content length exceeded 5246976 bytes.

In [ ]:
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# Visualization of the results of a detection.
vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict['detection_boxes'],
        output_dict['detection_classes'],
        output_dict['detection_scores'],
        category_index,
        instance_masks=output_dict.get('detection_masks'),
        use_normalized_coordinates=True,
        line_thickness=8)
plt.figure(figsize=IMAGE_SIZE)
plt.imshow(image_np)

Delete all temporary directories so that we are not affecting the next run. Also, optionally delete the end points.

In [ ]:
os.remove('model.tar.gz')
import shutil
shutil.rmtree('export')

If you do not want to continue using the endpoint, you can remove it. Remember, open endpoints are charged. If this is a simple test or practice, it is recommended to delete them.

In [17]:
sagemaker.Session().delete_endpoint(predictor.endpoint)

INFO:sagemaker:Deleting endpoint with name: sagemaker-tensorflow-py2-cpu-2018-03-15-02-28-39-225
